In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from imblearn.over_sampling import SMOTE


# XGB (class_weights)

In [1]:
# ---------------- Load the final dataframe ----------------
# write code below to load a dataframe in a directory not direclty in this directory
# ---------------- Load the final dataframe ----------------
# Ignore rows where Station == 'COLABA_-_IMD_OBSY_1854.0_7249.0'
df = pd.read_csv('station_outputs/DHARMABAD_1853.0_7750.0.csv')
#df = df[df['Station'] != 'COLABA_-_IMD_OBSY_1854.0_7249.0']
df = df.drop(columns=['Station'], errors='ignore')
df = df.drop(columns=['Date'], errors='ignore')

rainfall_threshold = df['Rainfall'].quantile(0.85)
df['Rainfall'] = (df['Rainfall'] >= rainfall_threshold).astype(int)

X = df.iloc[:, :-1]  
y = df.iloc[:, -1]   

# ---------------- Train-test split ----------------
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
scale_pos_weight = y_train.value_counts()[0] / y_train.value_counts()[1]
# ---------------- Train XGBoost model ----------------
model = XGBClassifier(
    n_estimators=100,
    max_depth=8,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    use_label_encoder=False,
    eval_metric='logloss',
    scale_pos_weight=scale_pos_weight
)

model.fit(X_train, y_train)

# ---------------- Evaluation ----------------
y_pred = model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))


NameError: name 'pd' is not defined

In [26]:
# print hit rate , false alarm rate and threat score from these metrics
def calculate_metrics(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    
    hit_rate = tp / (tp + fn) 
    false_alarm_rate = fp / (fp + tn) 
    threat_score = tp / (tp + fp + fn) 
    
    return hit_rate, false_alarm_rate, threat_score
hit_rate, false_alarm_rate, threat_score = calculate_metrics(y_test, y_pred)
print(f"\nHit Rate: {hit_rate:.2f}")
print(f"False Alarm Rate: {false_alarm_rate:.2f}")
print(f"Threat Score: {threat_score:.2f}")
# Save the model to a file


Hit Rate: 0.18
False Alarm Rate: 0.06
Threat Score: 0.14


In [24]:
# --- Predict and evaluate on a new file with ground truth ---

# Load the new data
new_df = pd.read_csv('overall_binarized_outputs/subset_18.9_72.8167.csv')

# Drop columns not needed for prediction
new_df = new_df.drop(columns=['station_lat', 'station_lon', 'month', 'dayofyear'], errors='ignore')

# Extract ground truth
y_true = new_df['Rainfall']

# Drop 'Rainfall' from features
X_new = new_df.drop(columns=['Rainfall'], errors='ignore')

# Predict using the trained model
y_pred = model.predict(X_new)

# Evaluate
print("Accuracy:", accuracy_score(y_true, y_pred))
print("\nClassification Report:\n", classification_report(y_true, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_true, y_pred))

# Optionally, add predictions to the dataframe and save
new_df['Predicted_Rainfall'] = y_pred
new_df.to_csv('predicted_results_with_truth.csv', index=False)
print(new_df.head())

# print hit rate , false alarm rate and threat score from these metrics
def calculate_metrics(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    
    hit_rate = tp / (tp + fn) 
    false_alarm_rate = fp / (fp + tn) 
    threat_score = tp / (tp + fp + fn) 
    
    return hit_rate, false_alarm_rate, threat_score
hit_rate, false_alarm_rate, threat_score = calculate_metrics(y_true, y_pred)
print(f"\nHit Rate: {hit_rate:.2f}")
print(f"False Alarm Rate: {false_alarm_rate:.2f}")
print(f"Threat Score: {threat_score:.2f}")

Accuracy: 0.8659058487874465

Classification Report:
               precision    recall  f1-score   support

           0       0.95      0.89      0.92       575
           1       0.60      0.78      0.68       126

    accuracy                           0.87       701
   macro avg       0.77      0.83      0.80       701
weighted avg       0.88      0.87      0.87       701


Confusion Matrix:
 [[509  66]
 [ 28  98]]
   R_H_L100_00_f018  R_H_L100_00_f021  R_H_L100_00_f024  R_H_L100_00_f027  \
0         96.707827         97.804086         96.366209         96.530411   
1         81.075206         86.335429         89.188677         87.674318   
2         85.684209         92.796396         97.554642         92.261322   
3         96.614821         87.513717         98.047336         96.483134   
4         98.052580         96.079917         95.405591         94.543160   

   R_H_L100_00_f030  R_H_L100_00_f033  R_H_L100_00_f036  R_H_L100_00_f039  \
0         97.645203         98.64117

# XGB (SMOTE)

In [ ]:

# ---------------- Load the final dataframe ----------------
df = pd.read_csv('combined_interpolated_data.csv')
df = df.drop(columns=['Station'], errors='ignore')
df = df.drop(columns=['Date'], errors='ignore')

rainfall_threshold = df['Rainfall'].quantile(0.85)
df['Rainfall'] = (df['Rainfall'] >= rainfall_threshold).astype(int)

X = df.iloc[:, :-1]  
y = df.iloc[:, -1]   
scale_pos_weight = y_train.value_counts()[0] / y_train.value_counts()[1]

# ---------------- Train-test split ----------------
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# ---------------- Apply SMOTE only on training data ----------------
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

# ---------------- Train XGBoost model ----------------
model = XGBClassifier(
    n_estimators=100,
    max_depth=8,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    use_label_encoder=False,
    eval_metric='logloss'
)

model.fit(X_train_smote, y_train_smote)

# ---------------- Evaluation ----------------
y_pred = model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))


/home/manoj/Desktop/MH/.venv/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [11:28:13] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Accuracy: 0.8723252496433667

Classification Report:
               precision    recall  f1-score   support

           0       0.94      0.90      0.92      3573
           1       0.56      0.69      0.62       633

    accuracy                           0.87      4206
   macro avg       0.75      0.80      0.77      4206
weighted avg       0.89      0.87      0.88      4206


Confusion Matrix:
 [[3231  342]
 [ 195  438]]


In [5]:
# print hit rate , false alarm rate and threat score from these metrics
def calculate_metrics(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    
    hit_rate = tp / (tp + fn) 
    false_alarm_rate = fp / (fp + tn) 
    threat_score = tp / (tp + fp + fn) 
    
    return hit_rate, false_alarm_rate, threat_score
hit_rate, false_alarm_rate, threat_score = calculate_metrics(y_test, y_pred)
print(f"\nHit Rate: {hit_rate:.2f}")
print(f"False Alarm Rate: {false_alarm_rate:.2f}")
print(f"Threat Score: {threat_score:.2f}")
# Save the model to a file


Hit Rate: 0.69
False Alarm Rate: 0.10
Threat Score: 0.45


# RF

In [ ]:

# ---------------- Load the final dataframe ----------------
df = pd.read_csv('combined_interpolated_data.csv')
df = df.drop(columns=['Station'], errors='ignore')
df = df.drop(columns=['Date'], errors='ignore')

# ---------------- Label rainfall as binary ----------------
rainfall_threshold = df['Rainfall'].quantile(0.85)
df['Rainfall'] = (df['Rainfall'] >= rainfall_threshold).astype(int)

X = df.iloc[:, :-1]
y = df.iloc[:, -1]

# ---------------- Train-test split ----------------
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# ---------------- Train Random Forest model ----------------
model = RandomForestClassifier(
    n_estimators=100,
    max_depth=15,
    class_weight='balanced',  # handles imbalance by adjusting weights
    random_state=42,
    n_jobs=-1
)

model.fit(X_train, y_train)

# ---------------- Evaluation ----------------
y_pred = model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))


Accuracy: 0.889205896338564

Classification Report:
               precision    recall  f1-score   support

           0       0.93      0.95      0.94      3573
           1       0.65      0.57      0.61       633

    accuracy                           0.89      4206
   macro avg       0.79      0.76      0.77      4206
weighted avg       0.88      0.89      0.89      4206


Confusion Matrix:
 [[3379  194]
 [ 272  361]]


In [10]:
# print hit rate , false alarm rate and threat score from these metrics
def calculate_metrics(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    
    hit_rate = tp / (tp + fn) 
    false_alarm_rate = fp / (fp + tn) 
    threat_score = tp / (tp + fp + fn) 
    
    return hit_rate, false_alarm_rate, threat_score
hit_rate, false_alarm_rate, threat_score = calculate_metrics(y_test, y_pred)
print(f"\nHit Rate: {hit_rate:.2f}")
print(f"False Alarm Rate: {false_alarm_rate:.2f}")
print(f"Threat Score: {threat_score:.2f}")
# Save the model to a file


Hit Rate: 0.57
False Alarm Rate: 0.05
Threat Score: 0.44
